In [0]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

In [0]:
import unicodedata
import re
import numpy as np
import os
import io
import time

In [0]:
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip',
    origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True,
    cache_dir=',',
)

In [47]:
path_to_zip

'/tmp/.keras/datasets/spa-eng.zip'

In [48]:
os.path.dirname(os.path.dirname(os.path.abspath(__file__)))

NameError: ignored

In [0]:
path_to_file = os.path.dirname(path_to_zip) + 'spa-eng/spa.txt'

In [0]:
path_to_file

In [0]:
path_to_file = os.path.join(
    os.path.dirname(path_to_zip),
    'spa-eng',
    'spa.txt'
)

In [0]:
#Unicode To ASCII

In [0]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
    if unicodedata.category(c) != 'Mn')

In [0]:
#Example
unicode_to_ascii("À")

In [0]:
def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [49]:
en_sentence = u"May I borrow this book?"
sp_sentence = u"¿Puedo tomar prestado este libro?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence).encode('utf-8'))
print(preprocess_sentence(sp_sentence))

<start> may i borrow this book ? <end>
b'<start> \xc2\xbf puedo tomar prestado este libro ? <end>'
<start> ¿ puedo tomar prestado este libro ? <end>


In [50]:
type('abc'.encode('utf-8'))

bytes

In [0]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [0]:
lines = lines = io.open(path_to_file, encoding='UTF-8').read().strip().split('\n')


In [53]:
lines[0:10]

['Go.\tVe.',
 'Go.\tVete.',
 'Go.\tVaya.',
 'Go.\tVáyase.',
 'Hi.\tHola.',
 'Run!\t¡Corre!',
 'Run.\tCorred.',
 'Who?\t¿Quién?',
 'Fire!\t¡Fuego!',
 'Fire!\t¡Incendio!']

In [0]:
word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:10]]

In [55]:
word_pairs

[['<start> go . <end>', '<start> ve . <end>'],
 ['<start> go . <end>', '<start> vete . <end>'],
 ['<start> go . <end>', '<start> vaya . <end>'],
 ['<start> go . <end>', '<start> vayase . <end>'],
 ['<start> hi . <end>', '<start> hola . <end>'],
 ['<start> run ! <end>', '<start> corre ! <end>'],
 ['<start> run . <end>', '<start> corred . <end>'],
 ['<start> who ? <end>', '<start> ¿ quien ? <end>'],
 ['<start> fire ! <end>', '<start> fuego ! <end>'],
 ['<start> fire ! <end>', '<start> incendio ! <end>']]

In [56]:
list(zip(*word_pairs))

[('<start> go . <end>',
  '<start> go . <end>',
  '<start> go . <end>',
  '<start> go . <end>',
  '<start> hi . <end>',
  '<start> run ! <end>',
  '<start> run . <end>',
  '<start> who ? <end>',
  '<start> fire ! <end>',
  '<start> fire ! <end>'),
 ('<start> ve . <end>',
  '<start> vete . <end>',
  '<start> vaya . <end>',
  '<start> vayase . <end>',
  '<start> hola . <end>',
  '<start> corre ! <end>',
  '<start> corred . <end>',
  '<start> ¿ quien ? <end>',
  '<start> fuego ! <end>',
  '<start> incendio ! <end>')]

In [0]:
english, spanish = create_dataset(path_to_file,None)

In [58]:
len(english)

118964

In [59]:
len(spanish)

118964

In [0]:
def max_length(tensor):
  return max(len(t) for t in tensor)

In [0]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      split=' ',#can be used other values too but since we used " " to split we are using nothing on split
      filters=''  
  )
  lang_tokenizer.fit_on_texts(lang)
  tensor = lang_tokenizer.texts_to_sequences(lang)
  tensor = tf.keras.preprocessing.sequence.pad_sequences(
      tensor,
      padding='post'
  )

  return tensor, lang_tokenizer

In [0]:
sample_tensor, english_tokkenize = tokenize(english)

In [63]:
type(sample_tensor)

numpy.ndarray

In [64]:
sample_tensor.shape

(118964, 51)

In [65]:
english[-1]

'<start> if you want to sound like a native speaker , you must be willing to practice saying the same sentence over and over in the same way that banjo players practice the same phrase over and over until they can play it correctly and at the desired tempo . <end>'

In [66]:
sample_tensor[-1:]

array([[    1,    72,     7,    39,     6,   718,    43,     9,   995,
         1277,    20,     7,   129,    37,  1426,     6,  1950,   586,
            5,   268,  1049,   181,    42,   181,    16,     5,   268,
          170,    17,  5242,  2246,  1950,     5,   268,  4623,   181,
           42,   181,   346,    48,    33,   200,    14,  3437,    42,
           44,     5, 12932, 12933,     3,     2]], dtype=int32)

In [68]:
sample_tensor[0]

array([ 1, 49,  3,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

In [0]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  targ_lang, inp_lang = create_dataset(path, num_examples) #changing here the languages can be converted into english to spanish or spanish to english

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [0]:
# Try experimenting with the size of that dataset
num_examples = 30000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [0]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor,target_tensor,test_size=0.2)

In [0]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [78]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
9 ----> el
16 ----> esta
517 ----> justo
920 ----> detras
14 ----> de
119 ----> ti
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
14 ----> he
11 ----> s
107 ----> right
670 ----> behind
6 ----> you
3 ----> .
2 ----> <end>


In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)